# Chatbot with LangChain conversational chain and OpenAI 🤖💬

<a target="_blank" href="https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/generative-ai/chatbot.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll build a chatbot that can respond to questions about custom data, such as policies of an employer.

The chatbot uses LangChain's `ConversationalRetrievalChain` and has the following capabilities:

- Answer questions asked in natural language
- Run hybrid search in Elasticsearch to find documents that answer the question
- Extract and summarize the answer using OpenAI LLM
- Maintain conversational memory for follow-up questions


## Requirements 🧰

For this example, you will need:

- Python 3.6 or later
- An Elastic deployment
  - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))
- OpenAI account

### Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, follow these steps to create one.

1. Go to [Elastic cloud Registration](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) and sign up for a free trial
2. Select **Create Deployment** and follow the instructions


## Install packages 📦

First we `pip install` the packages we need for this example.


In [2]:
%pip install -qU langchain openai elasticsearch tiktoken


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize clients 🔌

Next we input credentials with `getpass`. `getpass` is part of the Python standard library and is used to securely prompt for credentials.


In [3]:
from getpass import getpass

ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_USERNAME = getpass("Elastic username: ")
ELASTIC_PASSWORD = getpass("Elastic password: ")
OPENAI_API_KEY = getpass("OpenAI API key: ")

## Load and process documents 📄

Time to load some data! We'll be using the workplace search example data, which is a list of employee documents and policies.


In [4]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/generative-ai/data/workplace-docs.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

print(f"Successfully loaded {len(workplace_docs)} documents")

Successfully loaded 15 documents


## Chunk documents into passages 🪓

As we're chatting with our bot, it will run semantic searches on the index to find the relevant documents. In order for this to be accurate, we need to split the full documents into small chunks (also called passages). This way the semantic search will find the passage within a document that most likely answers our question.

We'll use LangChain's `CharacterTextSplitter` and split the documents' text at 800 characters with some overlap between chunks.


In [5]:
from langchain.text_splitter import CharacterTextSplitter

metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["content"])
    metadata.append({
        "name": doc["name"],
        "summary": doc["summary"]
    })

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512,
    chunk_overlap=256,
)
docs = text_splitter.create_documents(content, metadatas=metadata)

print(f"Split {len(workplace_docs)} documents into {len(docs)} passages")

Split 15 documents into 24 passages


Let's generate the embeddings and index the documents with them.


In [6]:
from langchain.vectorstores import ElasticsearchStore
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vector_store = ElasticsearchStore.from_documents(
    docs,
    es_cloud_id=ELASTIC_CLOUD_ID, 
    es_user=ELASTIC_USERNAME, 
    es_password=ELASTIC_PASSWORD,
    index_name="workplace-docs",
    embedding=embeddings
)

## Chat with the chatbot 💬

Let's initialize our chatbot. We'll define Elasticsearch as a store for retrieving documents and for storing the chat session history, OpenAI as the LLM to interpret questions and summarize answers, then we'll pass these to the conversational chain.


In [7]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ElasticsearchChatMessageHistory
from uuid import uuid4


retriever = vector_store.as_retriever()

llm = OpenAI(openai_api_key=OPENAI_API_KEY)

chat = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

session_id = str(uuid4())
chat_history = ElasticsearchChatMessageHistory(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_user=ELASTIC_USERNAME,
    es_password=ELASTIC_PASSWORD,
    session_id=session_id,
    index="workplace-docs-chat-history"
)

Now we can ask questions from our chatbot!

See how the chat history is passed as context for each question.


In [8]:
# Define a convenience function for Q&A
def ask(question, chat_history):
    result = chat({"question": question, "chat_history": chat_history.messages})
    print(f"""[QUESTION] {question}
[ANSWER]  {result["answer"]}
          [SUPPORTING DOCUMENTS] {list(map(lambda d: d.metadata["name"], list(result["source_documents"])))}""")
    chat_history.add_user_message(result["question"])
    chat_history.add_ai_message(result["answer"])

# Chat away!
print(f"[CHAT SESSION ID] {session_id}")
ask("What does NASA stand for?", chat_history)
ask("Which countries are part of it?", chat_history)
ask("Who are the team's leads?", chat_history)

[CHAT SESSION ID] 928a7f6a-cd02-4a4d-99a9-13e3486bb6e6
[QUESTION] What does NASA stand for?
[ANSWER]   NASA stands for North America South America region.
          [SUPPORTING DOCUMENTS] ['Sales Organization Overview', 'Intellectual Property Policy', 'April Work From Home Update', 'New Employee Onboarding Guide']
[QUESTION] Which countries are part of it?
[ANSWER]   The North America South America region includes the United States, Canada, Mexico, as well as Central and South America.
          [SUPPORTING DOCUMENTS] ['Sales Organization Overview', 'Wfh Policy Update May 2023', 'Fy2024 Company Sales Strategy', 'Fy2024 Company Sales Strategy']
[QUESTION] Who are the team's leads?
[ANSWER]   The lead of the North America South America region team is Laura Martinez, the Area Vice President of North America, and Gary Johnson, the Area Vice President of South America.
          [SUPPORTING DOCUMENTS] ['Sales Organization Overview', 'Fy2024 Company Sales Strategy', 'Wfh Policy Update May 20

💡 _Try experimenting with other questions or after clearing the workplace data, and observe how the responses change._


# (Optional) Clean up 🧹

Once we're done, we can clean up the chat history for this session...


In [8]:
chat_history.clear()

... or delete the indices.


In [11]:
vector_store.client.indices.delete(index='workplace-docs')
vector_store.client.indices.delete(index='workplace-docs-chat-history')

ObjectApiResponse({'acknowledged': True})